In [1]:
from azureml.core import Workspace, Datastore, Dataset, Experiment, ScriptRunConfig, Environment

ws = Workspace.from_config('./config.json')
az_store = Datastore.get(ws, 'from_sdk')
az_ds = Dataset.get_by_name(ws, 'Loan Applications Using SDK')
az_default_store = ws.get_default_datastore()

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cryptography 3.1.1 (c:\programdata\anaconda3\lib\site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).


In [2]:
new_exp = Experiment(workspace=ws, name='Loan_pipeline')

myenv = Environment(name='sklearn_env')

from azureml.core.environment import CondaDependencies
myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "sklearn_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"


In [13]:
## Create Pipeline steps
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineData
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute

dataFolder = PipelineData('datafoler', datastore=az_store)

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2, vm_priority='lowpriority')

compute_cluster = AmlCompute.create(workspace=ws, name='azml-cc-001', provisioning_configuration=compute_config)
compute_cluster.wait_for_completion()

run_config = RunConfiguration()
run_config.target = compute_cluster
run_config.environment = myenv

# Step 01 - Data Preparation
dataPrep_step = PythonScriptStep(name='01 Data Preparation', source_directory='.', 
                                 script_name='data-prep-pipeline.py', inputs=[az_ds.as_named_input('loan_dataset')],
                                 outputs=[dataFolder], runconfig=run_config, arguments=['--datafolder', dataFolder])

# Step 02 - Train and Predict
train_step = PythonScriptStep(name='02 Train', source_directory='.', 
                                 script_name='train-pipeline.py', inputs=[dataFolder],
                                 runconfig=run_config, arguments=['--datafolder', dataFolder])

# Configure and build the pipeline
from azureml.pipeline.core import Pipeline
new_pipeline = Pipeline(workspace=ws, steps=[dataPrep_step, train_step])

# Create an experiment
from azureml.core import Experiment
new_exp = Experiment(workspace=ws, name='MyPipeline')

new_pipeline_run = new_exp.submit(config=new_pipeline)

new_pipeline_run.wait_for_completion(show_output=True)

ully mounted azureml-blobstore-88484cd1-a3cf-4cbc-80a8-7884c76970d4 container from dp1002314018545 account at /mnt/batch/tasks/shared/LS_root/jobs/dp-100/azureml/e63a42f8-d4cf-4dc0-bcb6-35b77bc895b3/mounts/workspaceblobstore
>>>   2021/02/21 08:34:05 Mounting azml container from dp1002314018545 account at /mnt/batch/tasks/shared/LS_root/jobs/dp-100/azureml/e63a42f8-d4cf-4dc0-bcb6-35b77bc895b3/mounts/from_sdk
>>>   2021/02/21 08:34:05 Using Compute Identity to authenticate Blobfuse: false.
>>>   2021/02/21 08:34:05 Using Compute Identity to authenticate Blobfuse: false.
>>>   2021/02/21 08:34:05 Blobfuse cache size set to 9055 MB.
>>>   2021/02/21 08:34:05 Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/dp-100/azureml/e63a42f8-d4cf-4dc0-bcb6-35b77bc895b3/mounts/from_sdk --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/dp-100/azureml/e63a42f8-d4cf-4dc0-bcb6-35b77bc895b3/caches/from_sdk --file-cache-timeout-in-seconds=1000000 --cache-size-mb=9055 

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code",
        "messageFormat": "{Message}",
        "messageParameters": {
            "Message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code"
        },
        "details": [],
        "innerError": {
            "code": "BadArgument",
            "innerError": {
                "code": "AmlComputeBadRequest"
            }
        }
    },
    "correlation": {
        "operation": null,
        "request": "e486e61358ed1c11"
    },
    "environment": "southeastasia",
    "location": "southeastasia",
    "time": "2021-02-21T08:34:38.557141Z",
    "compon